# Get Object Data

In [1]:
import re

path = 'Google Maps Scraper/'

scraper_data = list()
excel_data = list()

with open(path + 'archivedurls.txt', 'r', encoding='utf8') as in_file1, open(path + 'scrapedlist.txt', 'r', encoding='utf8') as in_file2:
    for line in in_file1:
        scraper_data.append(re.sub('\n', '', line))

    for line in in_file2:
        excel_data.append(re.sub('\n', '', line))

# Select Object List

In [2]:
# input total finished object 
done = 162

In [3]:
# text for importing modules
last_name = ''

for name in excel_data[done:]:
    if name == last_name:
        continue
    last_name = name
    print('import '+re.sub('[ -.]', '_', name).lower())
    print('import_submodules('+re.sub('[ -.]', '_', name).lower()+')')

import pantai_ria_kenjeran
import_submodules(pantai_ria_kenjeran)


# Application

In [4]:
%pwd

'C:\\Users\\EMAR\\Tugas Akhir'

In [5]:
%cd "Google Maps Reviews"

C:\Users\EMAR\Tugas Akhir\Google Maps Reviews


In [6]:
from datetime import datetime
import os, os.path
import importlib
import pkgutil
import csv
import re

HEADER_ONE = ['id_review', 'caption', 'relative_date', 'retrieval_date', 'rating', 'username']
HEADER_TWO = ['object_name', 'gmaps_name', 'n_input', 'reviews_count', 'not_empty_count', 'url']

def csv_writer(mode, outfile, path='data/'):
    targetfile = open(path + outfile + '.csv', mode='w', encoding='utf-8', newline='\n')
    writer = csv.writer(targetfile, quoting=csv.QUOTE_MINIMAL)
    if mode == 1:
        h = HEADER_ONE
    else:
        h = HEADER_TWO
    writer.writerow(h)

    return writer

def parse(array):
    item = {}
    caption = ''
    
    if array[3] != None:
        caption = re.sub('\n', ' ', array[3])
    else:
        caption = ''

    item['id_review'] = array[10]
    item['caption'] = caption
    item['relative_date'] = array[1]
    item['retrieval_date'] = datetime.now()
    item['rating'] = array[4]
    item['username'] = array[0][1]
    
    return item

def get_reviews(array):
    reviews = array[2]
    
    parsed_reviews = []
    for index, review in enumerate(reviews):
        parsed_reviews.append(parse(review))

    return parsed_reviews

def import_submodules(package, recursive=True):
    """ Import all submodules of a module, recursively, including subpackages

    :param package: package (name or actual module)
    :type package: str | module
    :rtype: dict[str, types.ModuleType]
    """
    if isinstance(package, str):
        package = importlib.import_module(package)
    results = {}
    for loader, name, is_pkg in pkgutil.walk_packages(package.__path__):
        full_name = package.__name__ + '.' + name
        results[full_name] = importlib.import_module(full_name)
        if recursive and is_pkg:
            results.update(import_submodules(full_name))
    
    return results
        
def convert(place):
    place = place + "/"
    zero = ""
    counter = 1
    doc_count = len([name for name in os.listdir(place) if os.path.isfile(os.path.join(place, name))])

    while counter <= doc_count:
        if counter == 10:
            zero += "0"
        if counter == 100:
            zero += "0"
        if counter == 1000:
            zero += "0"
        counter += 1

    counter = 1

    while counter <= doc_count:
        if counter == 10:
            zero = zero[1:]
        if counter == 100:
            zero = zero[1:]
        if counter == 1000:
            zero = zero[1:]

        with open (place + "f " + zero + str(counter) + ".txt", "r", errors="ignore") as myfile:
            text = myfile.read()

        text = text[5:]
        text = re.sub('\n', '\n\t', text)
        first_line = "null = None\ndef get_array():\n\tarray = "
        last_line = "return array"
        text = first_line + text + last_line

        with open (place + "file" + str(counter) + ".py", "w", errors="ignore", encoding='utf8') as myfile:
            myfile.write(text)

        print("converting document " + str(counter))
        counter += 1
        
def write(place):
    writer = csv_writer(1, place)
    counter = 1
    empty_counter = 0
    doc_count = len([name for name in os.listdir(place) if os.path.isfile(os.path.join(place, name))])/2
    
    statistics = {}
    seen = set()

    while counter <= doc_count:
        module = place + '.file' + str(counter)
        reviews = get_reviews(eval(module).get_array())
        
        for r in reviews:
            row_data = list(r.values())
            id_review = row_data[0]
            caption = row_data[1]
            if id_review in seen: 
                continue
            if caption == "":
                empty_counter += 1
                
            seen.add(id_review)
            writer.writerow(row_data)

        print("writing from file " + str(counter))
        counter += 1
    
    unique = len(seen)
    not_empty = unique - empty_counter
    
    statistics['reviews_count'] = unique
    statistics['not_empty_count'] = not_empty
    
    return statistics

def backup_statistics(path='data/', place='statistics'):
    with open(path + place + '_combined.csv', 'r', encoding='utf8') as in_file, open(path + place + '_old.csv', 'w', encoding='utf8') as out_file:
        for line in in_file:
            out_file.write(line)
            
def combine_statistics(path='data/', place='statistics'):
    with open(path + place + '_old.csv', 'r', encoding='utf8') as in_file_1, open(path + place + '.csv', 'r', encoding='utf8') as in_file_2, open(path + place + '_combined.csv', 'w', encoding='utf8') as out_file:
        for file_1_line in in_file_1:
            out_file.write(file_1_line)
        for file_2_line in in_file_2:
            if file_2_line.startswith('object_name'):
                continue
            out_file.write(file_2_line)

def get_statistics(excel_list, scraper_list, statistics_list):
    parsed_data = []

    for index, name in enumerate(excel_list):
        place = re.sub('[ -.]', '_', name).lower()
        
        statistics_data = statistics_list[index]
        stats = list(statistics_data.values())
        unique = stats[0]
        not_empty = stats[1]
        
        scrape = scraper_list[index]
        data = scrape.split(',')
        url = data[0]
        n = data[1]
        gmaps_name = data[2]

        item = {}
        item['object_name'] = name
        item['gmaps_name'] = gmaps_name
        item['n_input'] = n
        item['reviews_count'] = unique
        item['not_empty_count'] = not_empty
        item['url'] = url

        parsed_data.append(item)
        print(name+" added to csv")
    
    return parsed_data

def delete_py(place):
    counter = 0
    folder_name = os.listdir(place)

    for file in folder_name:
        if file.endswith(".py"):
            os.remove(os.path.join(place, file))
            counter += 1

    print(str(counter) + " files deleted from " + place)
    
def delete_py_cache(place):
    counter = 0
    path = '/__pycache__'
    folder_name = os.listdir(place + path)

    for file in folder_name:
        if file.endswith(".pyc"):
            os.remove(os.path.join(place + path, file))
            counter += 1

    print(str(counter) + " cache files deleted from " + place)

# Make Folder

In [38]:
# for name in excel_data:
#     folder_name = re.sub('[ -.]', '_', name).lower()
#     if not os.path.exists(folder_name):
#         os.makedirs(folder_name)

# Convert txt to py

In [19]:
# multiple use
last_name = ''

for name in excel_data[done:]:
    if name == last_name:
        continue
    last_name = name
    place = re.sub('[ -.]', '_', name).lower()
    print("converting: " + place)
    convert(place)

converting: museum_kesehatan_dr.adhyatma
converting document 1
converting document 2
converting document 3
converting document 4
converting document 5
converting document 6
converting document 7
converting document 8
converting document 9
converting document 10
converting document 11
converting document 12
converting document 13
converting document 14
converting document 15
converting document 16
converting document 17
converting document 18
converting document 19
converting document 20
converting document 21
converting document 22
converting document 23
converting document 24
converting: museum_loka_jala_crana
converting document 1
converting document 2
converting document 3
converting document 4
converting document 5
converting document 6
converting document 7
converting document 8
converting document 9
converting document 10
converting document 11
converting document 12
converting document 13
converting document 14
converting document 15
converting document 16
converting: museum_pen

converting document 225
converting document 226
converting document 227
converting document 228
converting document 229
converting document 230
converting document 231
converting document 232
converting document 233
converting document 234
converting document 235
converting document 236
converting document 237
converting document 238
converting document 239
converting document 240
converting document 241
converting document 242
converting document 243
converting document 244
converting document 245
converting document 246
converting document 247
converting document 248
converting document 249
converting document 250
converting document 251
converting document 252
converting document 253
converting document 254
converting document 255
converting document 256
converting document 257
converting document 258
converting document 259
converting document 260
converting document 261
converting document 262
converting document 263
converting document 264
converting document 265
converting docum

converting document 577
converting document 578
converting document 579
converting document 580
converting document 581
converting document 582
converting document 583
converting document 584
converting document 585
converting document 586
converting document 587
converting document 588
converting document 589
converting document 590
converting document 591
converting document 592
converting document 593
converting document 594
converting document 595
converting document 596
converting document 597
converting document 598
converting document 599
converting document 600
converting document 601
converting document 602
converting document 603
converting document 604
converting document 605
converting document 606
converting document 607
converting document 608
converting document 609
converting document 610
converting document 611
converting document 612
converting document 613
converting document 614
converting document 615
converting document 616
converting document 617
converting docum

converting document 167
converting document 168
converting document 169
converting document 170
converting document 171
converting document 172
converting document 173
converting document 174
converting document 175
converting document 176
converting document 177
converting document 178
converting document 179
converting document 180
converting document 181
converting document 182
converting document 183
converting document 184
converting document 185
converting document 186
converting document 187
converting document 188
converting document 189
converting document 190
converting document 191
converting document 192
converting document 193
converting document 194
converting document 195
converting document 196
converting document 197
converting document 198
converting document 199
converting document 200
converting document 201
converting document 202
converting document 203
converting document 204
converting document 205
converting document 206
converting document 207
converting docum

converting document 126
converting document 127
converting document 128
converting document 129
converting document 130
converting document 131
converting document 132
converting document 133
converting document 134
converting document 135
converting document 136
converting document 137
converting document 138
converting document 139
converting document 140
converting document 141
converting document 142
converting document 143
converting document 144
converting document 145
converting document 146
converting document 147
converting document 148
converting document 149
converting document 150
converting document 151
converting document 152
converting document 153
converting document 154
converting document 155
converting document 156
converting document 157
converting document 158
converting document 159
converting document 160
converting document 161
converting document 162
converting document 163
converting document 164
converting document 165
converting document 166
converting docum

...

converting document 415
converting document 416
converting document 417
converting document 418
converting document 419
converting document 420
converting document 421
converting document 422
converting document 423
converting document 424
converting document 425
converting document 426
converting document 427
converting document 428
converting document 429
converting document 430
converting document 431
converting document 432
converting document 433
converting document 434
converting document 435
converting document 436
converting document 437
converting document 438
converting document 439
converting document 440
converting document 441
converting document 442
converting document 443
converting document 444
converting document 445
converting document 446
converting document 447
converting document 448
converting document 449
converting document 450
converting document 451
converting document 452
converting document 453
converting document 454
converting document 455
converting docum

converting document 299
converting document 300
converting document 301
converting document 302
converting document 303
converting document 304
converting document 305
converting document 306
converting document 307
converting document 308
converting document 309
converting document 310
converting document 311
converting document 312
converting document 313
converting document 314
converting document 315
converting document 316
converting document 317
converting document 318
converting document 319
converting document 320
converting document 321
converting document 322
converting document 323
converting document 324
converting document 325
converting document 326
converting document 327
converting document 328
converting document 329
converting document 330
converting document 331
converting document 332
converting document 333
converting document 334
converting document 335
converting document 336
converting document 337
converting document 338
converting document 339
converting docum

converting document 648
converting document 649
converting document 650
converting document 651
converting document 652
converting document 653
converting document 654
converting document 655
converting document 656
converting document 657
converting document 658
converting document 659
converting document 660
converting document 661
converting document 662
converting document 663
converting document 664
converting document 665
converting document 666
converting document 667
converting document 668
converting document 669
converting document 670
converting document 671
converting document 672
converting: soto_lamongan_cak_har
converting document 1
converting document 2
converting document 3
converting document 4
converting document 5
converting document 6
converting document 7
converting document 8
converting document 9
converting document 10
converting document 11
converting document 12
converting document 13
converting document 14
converting document 15
converting document 16
convert

converting document 322
converting document 323
converting document 324
converting document 325
converting document 326
converting document 327
converting document 328
converting document 329
converting document 330
converting document 331
converting document 332
converting document 333
converting document 334
converting document 335
converting document 336
converting document 337
converting document 338
converting document 339
converting document 340
converting document 341
converting document 342
converting document 343
converting document 344
converting document 345
converting document 346
converting document 347
converting document 348
converting document 349
converting document 350
converting document 351
converting document 352
converting document 353
converting document 354
converting document 355
converting document 356
converting document 357
converting document 358
converting document 359
converting document 360
converting document 361
converting document 362
converting docum

converting document 666
converting document 667
converting document 668
converting document 669
converting document 670
converting document 671
converting document 672
converting document 673
converting document 674
converting document 675
converting document 676
converting document 677
converting document 678
converting document 679
converting document 680
converting document 681
converting document 682
converting document 683
converting document 684
converting document 685
converting document 686
converting document 687
converting document 688
converting document 689
converting document 690
converting document 691
converting document 692
converting document 693
converting document 694
converting document 695
converting document 696
converting document 697
converting document 698
converting document 699
converting document 700
converting document 701
converting document 702
converting document 703
converting document 704
converting document 705
converting document 706
converting docum

In [7]:
# single use
place = "pantai_ria_kenjeran"
convert(place)

converting document 1
converting document 2
converting document 3
converting document 4
converting document 5
converting document 6
converting document 7
converting document 8
converting document 9
converting document 10
converting document 11
converting document 12
converting document 13
converting document 14
converting document 15
converting document 16
converting document 17


# Import Modules
Might take VERY long time

In [53]:
# 1-5
import taman_bungkul
import_submodules(taman_bungkul)
import taman_pelangi
import_submodules(taman_pelangi)
import monkasel
import_submodules(monkasel)
import tugu_pahlawan
import_submodules(tugu_pahlawan)
import tunjungan_plaza
import_submodules(tunjungan_plaza)

{'tunjungan_plaza.file1': <module 'tunjungan_plaza.file1' from 'C:\\Users\\EMAR\\Google Maps Reviews\\tunjungan_plaza\\file1.py'>,
 'tunjungan_plaza.file10': <module 'tunjungan_plaza.file10' from 'C:\\Users\\EMAR\\Google Maps Reviews\\tunjungan_plaza\\file10.py'>,
 'tunjungan_plaza.file100': <module 'tunjungan_plaza.file100' from 'C:\\Users\\EMAR\\Google Maps Reviews\\tunjungan_plaza\\file100.py'>,
 'tunjungan_plaza.file101': <module 'tunjungan_plaza.file101' from 'C:\\Users\\EMAR\\Google Maps Reviews\\tunjungan_plaza\\file101.py'>,
 'tunjungan_plaza.file102': <module 'tunjungan_plaza.file102' from 'C:\\Users\\EMAR\\Google Maps Reviews\\tunjungan_plaza\\file102.py'>,
 'tunjungan_plaza.file103': <module 'tunjungan_plaza.file103' from 'C:\\Users\\EMAR\\Google Maps Reviews\\tunjungan_plaza\\file103.py'>,
 'tunjungan_plaza.file104': <module 'tunjungan_plaza.file104' from 'C:\\Users\\EMAR\\Google Maps Reviews\\tunjungan_plaza\\file104.py'>,
 'tunjungan_plaza.file105': <module 'tunjungan_pla

In [24]:
# 6-51
import aiola_eatery
import_submodules(aiola_eatery)
import arca_joko_dolog
import_submodules(arca_joko_dolog)
import air_mancur_menari
import_submodules(air_mancur_menari)
import almond_crispy_wisata_rasa
import_submodules(almond_crispy_wisata_rasa)
import atlantis_land
import_submodules(atlantis_land)
import jembatan_suroboyo
import_submodules(jembatan_suroboyo)
import jembatan_ujung_galuh
import_submodules(jembatan_ujung_galuh)
import chipmunks_playland_and_cafe
import_submodules(chipmunks_playland_and_cafe)
import gedung_cerutu
import_submodules(gedung_cerutu)
import gedung_internatio
import_submodules(gedung_internatio)
import gedung_grahadi
import_submodules(gedung_grahadi)
import jalan_gula_dan_karet
import_submodules(jalan_gula_dan_karet)
import hutan_kota_balas_klumprik
import_submodules(hutan_kota_balas_klumprik)
import gado_gado_arjuno_suroboyo
import_submodules(gado_gado_arjuno_suroboyo)
import de_javasche_bank
import_submodules(de_javasche_bank)
import balai_pemuda
import_submodules(balai_pemuda)
import bebek_goreng_palupi
import_submodules(bebek_goreng_palupi)
import cakcuk_cafe_surabaya
import_submodules(cakcuk_cafe_surabaya)
import cagar_budaya_cak_durasim
import_submodules(cagar_budaya_cak_durasim)
import ciputra_waterpark
import_submodules(ciputra_waterpark)
import darmo_trade_centre
import_submodules(darmo_trade_centre)
import danau_unesa_lidah_wetan
import_submodules(danau_unesa_lidah_wetan)
import depot_pak_qomar_ayam_dan_bebek_goreng
import_submodules(depot_pak_qomar_ayam_dan_bebek_goreng)
import es_krim_zangrandi
import_submodules(es_krim_zangrandi)
import food_festival_pakuwon_city
import_submodules(food_festival_pakuwon_city)
import food_junction_grand_pakuwon
import_submodules(food_junction_grand_pakuwon)
import g_walk
import_submodules(g_walk)
import galaxy_mall
import_submodules(galaxy_mall)
import gedung_siola
import_submodules(gedung_siola)
import grand_city
import_submodules(grand_city)
import hotel_majapahit
import_submodules(hotel_majapahit)
import house_of_sampoerna
import_submodules(house_of_sampoerna)
import hutan_bambu_surabaya
import_submodules(hutan_bambu_surabaya)
import jembatan_merah
import_submodules(jembatan_merah)
import kebun_bibit_wonorejo
import_submodules(kebun_bibit_wonorejo)
import kebun_binatang_surabaya
import_submodules(kebun_binatang_surabaya)
import kenjeran_park
import_submodules(kenjeran_park)
import kepiting_kapasan_surabaya
import_submodules(kepiting_kapasan_surabaya)
import klenteng_sanggar_agung
import_submodules(klenteng_sanggar_agung)
import lenmarc_mall
import_submodules(lenmarc_mall)
import masjid_al_akbar_surabaya
import_submodules(masjid_al_akbar_surabaya)
import masjid_ampel
import_submodules(masjid_ampel)
import masjid_chengho_surabaya
import_submodules(masjid_chengho_surabaya)
import monumen_bambu_runcing
import_submodules(monumen_bambu_runcing)
import monumen_jalasveva_jayamahe
import_submodules(monumen_jalasveva_jayamahe)
import monumen_perjuangan_polisi_republik_indonesia
import_submodules(monumen_perjuangan_polisi_republik_indonesia)

{}

In [25]:
#52-94
import museum_kesehatan_dr_adhyatma
import_submodules(museum_kesehatan_dr_adhyatma)
import museum_loka_jala_crana
import_submodules(museum_loka_jala_crana)
import museum_pendidikan_surabaya
import_submodules(museum_pendidikan_surabaya)
import museum_sepuluh_nopember
import_submodules(museum_sepuluh_nopember)
import nasi_empal_pengampon
import_submodules(nasi_empal_pengampon)
import pakuwon_mall
import_submodules(pakuwon_mall)
import pakuwon_trade_centre
import_submodules(pakuwon_trade_centre)
import pasar_atom
import_submodules(pasar_atom)
import patung_buddha_4_rupa
import_submodules(patung_buddha_4_rupa)
import patung_suro_dan_boyo
import_submodules(patung_suro_dan_boyo)
import rawon_setan
import_submodules(rawon_setan)
import rolag_kopi_karah
import_submodules(rolag_kopi_karah)
import royal_plaza
import_submodules(royal_plaza)
import rujak_cingur_ahmad_jais
import_submodules(rujak_cingur_ahmad_jais)
import rumah_hantu_darmo
import_submodules(rumah_hantu_darmo)
import sate_karak
import_submodules(sate_karak)
import sate_lisidu
import_submodules(sate_lisidu)
import sate_klopo_ondomohen
import_submodules(sate_klopo_ondomohen)
import sego_sambel_mak_yeye
import_submodules(sego_sambel_mak_yeye)
import stadion_gelora_bung_tomo
import_submodules(stadion_gelora_bung_tomo)
import koni
import_submodules(koni)
import spazio_surabaya
import_submodules(spazio_surabaya)
import spikoe_resep_kuno
import_submodules(spikoe_resep_kuno)
import surabaya_north_quay
import_submodules(surabaya_north_quay)
import surabaya_patata
import_submodules(surabaya_patata)
import tahu_campur_kalasan
import_submodules(tahu_campur_kalasan)
import tahu_telor_pak_jayen
import_submodules(tahu_telor_pak_jayen)
import makam_peneleh
import_submodules(makam_peneleh)
import monumen_parasamya_purnakarya_nugraha
import_submodules(monumen_parasamya_purnakarya_nugraha)
import monumen_yos_sudarso
import_submodules(monumen_yos_sudarso)
import penjara_kalisosok
import_submodules(penjara_kalisosok)
import rumah_hantu_kupang
import_submodules(rumah_hantu_kupang)
import taman_makam_pahlawan_sepuluh_nopember
import_submodules(taman_makam_pahlawan_sepuluh_nopember)
import taman_apsari
import_submodules(taman_apsari)
import taman_balai_kota_surabaya
import_submodules(taman_balai_kota_surabaya)
import taman_korea
import_submodules(taman_korea)
import taman_kunang_kunang
import_submodules(taman_kunang_kunang)
import taman_prestasi
import_submodules(taman_prestasi)
import taman_skate_dan_bmx
import_submodules(taman_skate_dan_bmx)
import the_harvest_chocolate_and_patisierre
import_submodules(the_harvest_chocolate_and_patisierre)
import surabaya_night_carnival
import_submodules(surabaya_night_carnival)
import taman_cahaya
import_submodules(taman_cahaya)
import soto_ambengan_pak_sadi
import_submodules(soto_ambengan_pak_sadi)

{'taman_persahabatan.file1': <module 'taman_persahabatan.file1' from 'C:\\Users\\EMAR\\Google Maps Reviews\\taman_persahabatan\\file1.py'>,
 'taman_persahabatan.file10': <module 'taman_persahabatan.file10' from 'C:\\Users\\EMAR\\Google Maps Reviews\\taman_persahabatan\\file10.py'>,
 'taman_persahabatan.file11': <module 'taman_persahabatan.file11' from 'C:\\Users\\EMAR\\Google Maps Reviews\\taman_persahabatan\\file11.py'>,
 'taman_persahabatan.file12': <module 'taman_persahabatan.file12' from 'C:\\Users\\EMAR\\Google Maps Reviews\\taman_persahabatan\\file12.py'>,
 'taman_persahabatan.file2': <module 'taman_persahabatan.file2' from 'C:\\Users\\EMAR\\Google Maps Reviews\\taman_persahabatan\\file2.py'>,
 'taman_persahabatan.file3': <module 'taman_persahabatan.file3' from 'C:\\Users\\EMAR\\Google Maps Reviews\\taman_persahabatan\\file3.py'>,
 'taman_persahabatan.file4': <module 'taman_persahabatan.file4' from 'C:\\Users\\EMAR\\Google Maps Reviews\\taman_persahabatan\\file4.py'>,
 'taman_per

In [7]:
# 95-106
import bebek_goreng_sinjay
import_submodules(bebek_goreng_sinjay)
import depot_bu_rudy
import_submodules(depot_bu_rudy)
import soto_lamongan_cak_har
import_submodules(soto_lamongan_cak_har)
import brownies_amanda
import_submodules(brownies_amanda)
import lapis_kukus_pahlawan
import_submodules(lapis_kukus_pahlawan)
import nasi_goreng_jancuk_surabaya
import_submodules(nasi_goreng_jancuk_surabaya)
import rumah_batik
import_submodules(rumah_batik)
import lontong_balap_pak_gendut
import_submodules(lontong_balap_pak_gendut)
import soto_madura_tapak_siring
import_submodules(soto_madura_tapak_siring)
import spesial_belut_surabaya
import_submodules(spesial_belut_surabaya)
import taman_mangrove_wonorejo
import_submodules(taman_mangrove_wonorejo)
import taman_persahabatan
import_submodules(taman_persahabatan)

{'taman_persahabatan.file1': <module 'taman_persahabatan.file1' from 'C:\\Users\\EMAR\\Google Maps Reviews\\taman_persahabatan\\file1.py'>,
 'taman_persahabatan.file10': <module 'taman_persahabatan.file10' from 'C:\\Users\\EMAR\\Google Maps Reviews\\taman_persahabatan\\file10.py'>,
 'taman_persahabatan.file11': <module 'taman_persahabatan.file11' from 'C:\\Users\\EMAR\\Google Maps Reviews\\taman_persahabatan\\file11.py'>,
 'taman_persahabatan.file12': <module 'taman_persahabatan.file12' from 'C:\\Users\\EMAR\\Google Maps Reviews\\taman_persahabatan\\file12.py'>,
 'taman_persahabatan.file2': <module 'taman_persahabatan.file2' from 'C:\\Users\\EMAR\\Google Maps Reviews\\taman_persahabatan\\file2.py'>,
 'taman_persahabatan.file3': <module 'taman_persahabatan.file3' from 'C:\\Users\\EMAR\\Google Maps Reviews\\taman_persahabatan\\file3.py'>,
 'taman_persahabatan.file4': <module 'taman_persahabatan.file4' from 'C:\\Users\\EMAR\\Google Maps Reviews\\taman_persahabatan\\file4.py'>,
 'taman_per

In [8]:
#107
import pantai_ria_kenjeran
import_submodules(pantai_ria_kenjeran)

{'pantai_ria_kenjeran.file1': <module 'pantai_ria_kenjeran.file1' from 'C:\\Users\\EMAR\\Tugas Akhir\\Google Maps Reviews\\pantai_ria_kenjeran\\file1.py'>,
 'pantai_ria_kenjeran.file10': <module 'pantai_ria_kenjeran.file10' from 'C:\\Users\\EMAR\\Tugas Akhir\\Google Maps Reviews\\pantai_ria_kenjeran\\file10.py'>,
 'pantai_ria_kenjeran.file11': <module 'pantai_ria_kenjeran.file11' from 'C:\\Users\\EMAR\\Tugas Akhir\\Google Maps Reviews\\pantai_ria_kenjeran\\file11.py'>,
 'pantai_ria_kenjeran.file12': <module 'pantai_ria_kenjeran.file12' from 'C:\\Users\\EMAR\\Tugas Akhir\\Google Maps Reviews\\pantai_ria_kenjeran\\file12.py'>,
 'pantai_ria_kenjeran.file13': <module 'pantai_ria_kenjeran.file13' from 'C:\\Users\\EMAR\\Tugas Akhir\\Google Maps Reviews\\pantai_ria_kenjeran\\file13.py'>,
 'pantai_ria_kenjeran.file14': <module 'pantai_ria_kenjeran.file14' from 'C:\\Users\\EMAR\\Tugas Akhir\\Google Maps Reviews\\pantai_ria_kenjeran\\file14.py'>,
 'pantai_ria_kenjeran.file15': <module 'pantai_ri

# Write to CSV

In [11]:
# multiple use
statistics_data = []
for name in excel_data[done:]:
    place = re.sub('[ -.]', '_', name).lower()
    print("writing: "+place)
    statistics_data.append(write(place))

writing: pantai_ria_kenjeran
writing from file 1
writing from file 2
writing from file 3
writing from file 4
writing from file 5
writing from file 6
writing from file 7
writing from file 8
writing from file 9
writing from file 10
writing from file 11
writing from file 12
writing from file 13
writing from file 14
writing from file 15
writing from file 16
writing from file 17


In [10]:
# single use
# write(re.sub('[ -.]', '_', excel_data[163]).lower())

# Write Statistics

In [12]:
statistics_data[0]

{'reviews_count': 1669, 'not_empty_count': 838}

In [13]:
backup_statistics()

In [15]:
writer = csv_writer(2, 'statistics')

statistics = get_statistics(excel_data[done:], scraper_data[done:], statistics_data)

for r in statistics:
    row_data = list(r.values())
    writer.writerow(row_data)
    
combine_statistics()

Pantai Ria Kenjeran added to csv


# Delete Unused py Files

In [16]:
# don't run unless needed
last_name = ''

for name in excel_data:
    if name == last_name:
        continue
    last_name = name
    place = re.sub('[ -.]', '_', name).lower()
    delete_py(place) # delete python files
    delete_py_cache(place) # delete generated cache files

0 files deleted from taman_bungkul
0 cache files deleted from taman_bungkul
0 files deleted from taman_pelangi
0 cache files deleted from taman_pelangi
0 files deleted from monkasel
0 cache files deleted from monkasel
0 files deleted from tugu_pahlawan
0 cache files deleted from tugu_pahlawan
0 files deleted from tunjungan_plaza
0 cache files deleted from tunjungan_plaza
300 files deleted from aiola_eatery
0 cache files deleted from aiola_eatery
0 files deleted from arca_joko_dolog
0 cache files deleted from arca_joko_dolog
0 files deleted from air_mancur_menari
0 cache files deleted from air_mancur_menari
0 files deleted from almond_crispy_wisata_rasa
0 cache files deleted from almond_crispy_wisata_rasa
0 files deleted from atlantis_land
0 cache files deleted from atlantis_land
0 files deleted from jembatan_suroboyo
0 cache files deleted from jembatan_suroboyo
0 files deleted from jembatan_ujung_galuh
0 cache files deleted from jembatan_ujung_galuh
0 files deleted from chipmunks_playl

0 files deleted from depot_bu_rudy
0 cache files deleted from depot_bu_rudy
0 files deleted from soto_lamongan_cak_har
0 cache files deleted from soto_lamongan_cak_har
0 files deleted from brownies_amanda
0 cache files deleted from brownies_amanda
0 files deleted from lapis_kukus_pahlawan
0 cache files deleted from lapis_kukus_pahlawan
0 files deleted from nasi_goreng_jancuk_surabaya
0 cache files deleted from nasi_goreng_jancuk_surabaya
0 files deleted from rumah_batik
0 cache files deleted from rumah_batik
0 files deleted from lontong_balap_pak_gendut
0 cache files deleted from lontong_balap_pak_gendut
0 files deleted from soto_madura_tapak_siring
0 cache files deleted from soto_madura_tapak_siring
0 files deleted from spesial_belut_surabaya
0 cache files deleted from spesial_belut_surabaya
0 files deleted from taman_mangrove_wonorejo
0 cache files deleted from taman_mangrove_wonorejo
0 files deleted from taman_persahabatan
0 cache files deleted from taman_persahabatan
17 files delet

# Get Rid of Duplicates

In [17]:
# # old method
# with open('data/reviews.csv','r', encoding='utf8') as in_file, open('data/reviews_no_duplicate.csv','w', encoding='utf8') as out_file:
#     seen = set() # set for fast O(1) amortized lookup
#     for line in in_file:
#         id_review = line.split(',')[0]
#         if id_review in seen: continue # skip duplicate

#         seen.add(id_review)
#         out_file.write(line)

# Count Statistics and Save As

In [16]:
# path = 'data/'
# place = 'house_of_sampoerna'
# # old method
# # with open('data/reviews_no_duplicate.csv','r', encoding='utf8') as in_file, open('data/'+place+' '+str(len(seen)-1)+'.csv','w+', encoding='utf8') as out_file:
# with open(path+place+'.csv','r', encoding='utf8') as in_file:
#     counter = 0
#     lines = 0
#     for index, line in enumerate(in_file):
# #         out_file.write(line)
#         caption = line.split(',')[1]
#         print(index)
#         print(caption)
#         lines += 1
#         if caption == "":
#             counter += 1
            
# diff = lines-counter
# print("Amount of empty caption is "+str(counter)+" in "+str(lines)+" row. Resulting in only "+str(diff)+" not empty captions")
# print("Amount of unique date is "+str(len(seen)-1))

# Back to User/EMAR

In [4]:
%cd ..

C:\Users\EMAR


In [5]:
%pwd

'C:\\Users\\EMAR'